In [1]:
import pandas as pd
import syft as sy
import numpy as np
from syft.core.adp.entity import DataSubject

### Loading the dataset

In [2]:
data = pd.read_csv("../datasets/us - feb 2021.csv")[0:100]
data.head()

/Users/areopagus/opt/anaconda3/envs/syft/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Classification  Year  Period   Period Desc.  Aggregate Level  Is Leaf Code  \
0             HS  2021  202102  February 2021                4             0   
1             HS  2021  202102  February 2021                6             1   
2             HS  2021  202102  February 2021                6             1   
3             HS  2021  202102  February 2021                6             1   
4             HS  2021  202102  February 2021                6             1   

   Trade Flow Code Trade Flow  Reporter Code                  Reporter  ...  \
0                2    Exports            842  United States of America  ...   
1                1    Imports            842  United States of America  ...   
2                1    Imports            842  United States of America  ...   
3                1    Imports            842  United States of America  ...   
4                1    Imports            842  United States of America  ...   

     Partner  Partner ISO Commodity Code  \
0  Australia          NaN           2303   
1      World          NaN         100410   
2      World          NaN         100510   
3      World          NaN         100590   
4      World          NaN         100620   

                                           Commodity Qty Unit Code Qty Unit  \
0  Residues of starch manufacture, similar residu...             0      NaN   
1                               Cereals; oats, seeds             0      NaN   
2                        Cereals; maize (corn), seed             0      NaN   
3             Cereals; maize (corn), other than seed             0      NaN   
4                       Cereals; husked (brown) rice             0      NaN   

   Qty  Netweight (kg)  Trade Value (US$)  Flag  
0  NaN        651000.0             163488     0  
1  NaN       1582215.0             679144     0  
2  NaN       1183421.0            7090440     0  
3  NaN      21947487.0            6064085     0  
4  NaN       2620330.0            3343948     0  

[5 rows x 22 columns]

### Logging into the domain

In [3]:
usa = sy.login(email="info@openmined.org",
             password="changethis",
             port=8082)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to http://localhost:8082... done! 	 Logging into usa... done!


### Upload the dataset to Domain node

In [4]:
# We will upload only the first few rows
# All these three columns are of `int` type

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT. We need to create flags for this or something
data_batch = ((np.array(list(data['Trade Value (US$)'])) / 100000)[0:10]).astype(np.int32)
trade_partners = ((list(data['Partner'])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_usa_dataset = sy.Tensor(data_batch)
sampled_usa_dataset.public_shape = sampled_usa_dataset.shape
sampled_usa_dataset = sampled_usa_dataset.private(0, 3, entities="bob").tag("trade_flow")

In [5]:
usa.load_dataset(
    assets={"USA Trade": sampled_usa_dataset},
    name="USA Trade Data - First few rows",
    description="""A collection of reports from USA's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [6]:
usa.datasets

### Create a Data Scientist User

In [7]:
usa.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget":200
    }
)

### Accept/Deny Requests to the Domain

In [ ]:
usa.requests.pandas

In [ ]:
usa.requests[-1].accept()